In [0]:
%sql
USE CATALOG catalog_cp;
CREATE SCHEMA IF NOT EXISTS silver;
USE silver;

CREATE TABLE IF NOT EXISTS region (
    regionkey INT NOT NULL,
    name STRING NOT NULL,
    comment STRING,
    PRIMARY KEY (regionkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS nation (
    nationkey INT NOT NULL,
    name STRING NOT NULL,
    regionkey INT NOT NULL,
    comment STRING,
    PRIMARY KEY (nationkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS part (
    partkey INT NOT NULL,
    name STRING NOT NULL,
    mfgr STRING,
    brand STRING,
    type STRING,
    size INT,
    container STRING,
    retailprice DECIMAL(12,2),
    comment STRING,
    PRIMARY KEY (partkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS supplier (
    suppkey INT NOT NULL,
    name STRING NOT NULL,
    address STRING,
    nationkey INT NOT NULL,
    phone STRING,
    acctbal DECIMAL(12,2),
    comment STRING,
    PRIMARY KEY (suppkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS partsupp (
    partkey INT NOT NULL,
    suppkey INT NOT NULL,
    availqty INT,
    supplycost DECIMAL(12,2),
    comment STRING,
    PRIMARY KEY (partkey, suppkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS customer (
    custkey INT NOT NULL,
    name STRING NOT NULL,
    address STRING,
    nationkey INT NOT NULL,
    phone STRING,
    acctbal DECIMAL(12,2),
    mktsegment STRING,
    comment STRING,
    PRIMARY KEY (custkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS orders (
    orderkey INT NOT NULL,
    custkey INT NOT NULL,
    orderstatus STRING,
    totalprice DECIMAL(12,2),
    orderdate DATE,
    orderpriority STRING,
    clerk STRING,
    shippriority INT,
    comment STRING,
    PRIMARY KEY (orderkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS lineitem (
    orderkey INT NOT NULL,
    partkey INT NOT NULL,
    suppkey INT NOT NULL,
    linenumber INT NOT NULL,
    quantity DECIMAL(12,2),
    extendedprice DECIMAL(12,2),
    discount DECIMAL(12,2),
    tax DECIMAL(12,2),
    returnflag STRING,
    linestatus STRING,
    shipdate DATE,
    commitdate DATE,
    receiptdate DATE,
    shipinstruct STRING,
    shipmode STRING,
    comment STRING,
    PRIMARY KEY (orderkey, linenumber)
) USING DELTA;


In [0]:
%sql
USE CATALOG catalog_cp;
USE SCHEMA silver;

MERGE INTO region AS target
USING bronze.region AS source
ON target.r_regionkey = source.r_regionkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO nation AS target
USING bronze.nation AS source
ON target.n_nationkey = source.n_nationkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO part AS target
USING bronze.part AS source
ON target.p_partkey = source.p_partkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO supplier AS target
USING bronze.supplier AS source
ON target.s_suppkey = source.s_suppkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO partsupp AS target
USING bronze.partsupp AS source
ON target.ps_partkey = source.ps_partkey AND target.ps_suppkey = source.ps_suppkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO customer AS target
USING bronze.customer AS source
ON target.c_custkey = source.c_custkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO orders AS target
USING bronze.orders AS source
ON target.o_orderkey = source.o_orderkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO lineitem AS target
USING bronze.lineitem AS source
ON target.l_orderkey = source.l_orderkey AND target.l_linenumber = source.l_linenumber
WHEN NOT MATCHED THEN INSERT *;

In [0]:
%sql
USE CATALOG catalog_cp;

WITH bronze_counts AS (
  SELECT 'region' AS table_name, COUNT(*) AS row_count FROM bronze.region UNION ALL
  SELECT 'nation', COUNT(*) FROM bronze.nation UNION ALL
  SELECT 'part', COUNT(*) FROM bronze.part UNION ALL
  SELECT 'supplier', COUNT(*) FROM bronze.supplier UNION ALL
  SELECT 'partsupp', COUNT(*) FROM bronze.partsupp UNION ALL
  SELECT 'customer', COUNT(*) FROM bronze.customer UNION ALL
  SELECT 'orders', COUNT(*) FROM bronze.orders UNION ALL
  SELECT 'lineitem', COUNT(*) FROM bronze.lineitem
),

silver_counts AS (
  SELECT 'region' AS table_name, COUNT(*) AS row_count FROM silver.region UNION ALL
  SELECT 'nation', COUNT(*) FROM silver.nation UNION ALL
  SELECT 'part', COUNT(*) FROM silver.part UNION ALL
  SELECT 'supplier', COUNT(*) FROM silver.supplier UNION ALL
  SELECT 'partsupp', COUNT(*) FROM silver.partsupp UNION ALL
  SELECT 'customer', COUNT(*) FROM silver.customer UNION ALL
  SELECT 'orders', COUNT(*) FROM silver.orders UNION ALL
  SELECT 'lineitem', COUNT(*) FROM silver.lineitem
)

SELECT
  b.table_name,
  b.row_count AS bronze_count,
  s.row_count AS silver_count,
  CASE WHEN b.row_count = s.row_count THEN '✅ MATCH' ELSE '❌ MISMATCH' END AS status
FROM bronze_counts b
JOIN silver_counts s ON b.table_name = s.table_name
ORDER BY table_name;
